In [1]:
#Import Libraries
import pandas as pd
import string
from nltk.corpus import stopwords

In [2]:
# Load the spam data file
spam_data = pd.read_csv('C:\Users\pravinw\Documents\Fractal\DATA SCIENTIST\PractiseAssignment_Simplilearn\SpamCollection\SpamCollection', sep='\t', names=['Response','Message'])

In [3]:
# View top 5 records
spam_data.head()

,Response,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
# Describe the dataset
spam_data.describe()

,Response,Message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [5]:
# Group by Response
spam_data.groupby('Response').describe()

Message
Response                                                          
ham      count                                                4825
         unique                                               4516
         top                                Sorry, I'll call later
         freq                                                   30
spam     count                                                 747
         unique                                                653
         top     Please call our customer service representativ...
         freq                                                    4

In [6]:
# Check the length of each message
spam_data['Length'] = spam_data['Message'].apply(len)

In [7]:
spam_data.head()

,Response,Message,Length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
# Define function to filter out punctuations and stop words
def msg_text_process(msg):
    no_punctuation = [char for char in msg if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return[word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]

In [9]:
# View the output
spam_data['Message'].head(5).apply(msg_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: Message, dtype: object

In [10]:
# import library for text processing
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# use bag of words by applying the function and fit the data into it
bagofwords = CountVectorizer(analyzer=msg_text_process).fit(spam_data['Message'])

C:\Users\pravinw\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [12]:
# print length of bag of words stored in vocabulary_
print len(bagofwords.vocabulary_)

11425


In [13]:
msg_bagofwords = bagofwords.transform(spam_data['Message'])
print msg_bagofwords

C:\Users\pravinw\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


  (0, 1110)	1
  (0, 1483)	1
  (0, 2060)	1
  (0, 4653)	1
  (0, 5217)	1
  (0, 5218)	1
  (0, 5769)	1
  (0, 6217)	1
  (0, 6906)	1
  (0, 6937)	1
  (0, 7555)	1
  (0, 7668)	1
  (0, 8336)	1
  (0, 8917)	1
  (0, 10965)	1
  (0, 11163)	1
  (1, 2451)	1
  (1, 3064)	1
  (1, 7701)	1
  (1, 8590)	1
  (1, 10698)	1
  (1, 11072)	1
  (2, 73)	1
  (2, 423)	1
  (2, 430)	1
  :	:
  (5568, 6691)	1
  (5568, 6882)	1
  (5568, 7159)	1
  (5568, 11418)	1
  (5569, 3228)	1
  (5569, 3721)	1
  (5569, 8252)	1
  (5569, 10199)	1
  (5570, 4508)	1
  (5570, 5055)	1
  (5570, 5251)	1
  (5570, 6282)	1
  (5570, 6699)	1
  (5570, 6799)	1
  (5570, 6984)	1
  (5570, 7287)	1
  (5570, 7394)	1
  (5570, 7800)	1
  (5570, 8420)	1
  (5570, 9915)	1
  (5570, 10787)	1
  (5570, 11006)	1
  (5571, 3431)	1
  (5571, 8348)	1
  (5571, 10648)	1


In [14]:
# Import Libraries for tfidf using transformer fit bag of words
from sklearn.feature_extraction.text import TfidfTransformer
tdidf_transformer = TfidfTransformer().fit(msg_bagofwords)

In [15]:
# Check the number of rows and column in tfidf
message_tfidf = tdidf_transformer.transform(msg_bagofwords)
print message_tfidf.shape

(5572, 11425)


In [16]:
# choose naive bayes model to detect spam
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf, spam_data['Response'])

In [17]:
# Check model for predicted and expected value
message = spam_data['Message'][2]
bag_of_words_for_message = bagofwords.transform([message])
tfidf = tdidf_transformer.transform(bag_of_words_for_message)

print 'predicted: ', spam_detect_model.predict(tfidf)[0]
print 'expected: ', spam_data.Response[2]

predicted:  spam
expected:  spam
